In [1]:
using CSV
using Distributions
using StatsFuns

dfx = CSV.read("./Xvals.csv", delim=',', header=true)
dfy = CSV.read("./Yvals.csv", delim=',', header=true)
nb = size(dfx, 1);
A = convert(Matrix{Float64}, CSV.read("./affinitymatrix.csv", delim=',', header=true)[1:10, 2:11]);
X = convert(Matrix{Float64}, dfx)
Y = convert(Matrix{Float64}, dfy);

In [2]:
X = (X .- mean(X, 1)) ./ std(X, 1)
Y = (Y .- mean(Y, 1)) ./ std(Y, 1);

In [3]:
Φ = X * A * Y';

In [4]:
function iterate(Φ; σ = 1e-1, tol=1e-6)
    K = exp.(Φ/σ)
    nbx = size(Φ, 1)
    nby = size(Φ, 2)
    Ax = ones(nbx)/nbx
    By = ones(nby)/nby
    error = 1.
    iteration = 0

    #π = ones(nb, nb)*1/nb/nb
    #πold = ones(nb, nb)*1/nb/nb;

    Axold = copy(Ax)
    Byold = copy(By)

    while (error > tol)

        Ax = (1/nbx)./(K * By)
        By = (1/nby)./(Ax' * K)'

        #πold = copy(π)

        #error = maximum(abs.(π-πold))
        error = max(maximum(abs.(Ax - Axold)), maximum(abs.(By - Byold)))
        iteration += 1
        Axold = copy(Ax)
        Byold = copy(By)
    end

    π = Ax .* K .* By'
    return (π, sum(π .* Φ), iteration, error)
end

iterate (generic function with 1 method)

In [5]:
@time iterate(Φ)

  3.073720 seconds (416.82 k allocations: 96.653 MiB, 3.21% gc time)


([1.0315e-9 5.32028e-9 … 7.20743e-16 1.74851e-6; 5.80025e-8 1.14765e-6 … 7.19417e-9 8.83245e-8; … ; 1.67629e-9 2.71497e-10 … 1.89219e-7 1.43221e-7; 9.67748e-8 3.3422e-7 … 5.09165e-9 3.15713e-8], 1.5593130457556408, 416, 9.970700460826265e-7)

In [6]:
Φ = rand(Uniform(), 10, 8)

10×8 Array{Float64,2}:
 0.0769457  0.385665    0.279565  0.609245  …  0.21408    0.294209  0.683291 
 0.0780143  0.863369    0.508569  0.618796     0.269703   0.567243  0.0888409
 0.434742   0.409363    0.84595   0.45049      0.561631   0.577851  0.321521 
 0.330454   0.279709    0.115476  0.412075     0.0320698  0.656574  0.331258 
 0.808554   0.661124    0.403367  0.063296     0.412741   0.481697  0.723882 
 0.913206   0.00716369  0.792355  0.982216  …  0.743961   0.891388  0.4274   
 0.371863   0.671858    0.605177  0.315029     0.709756   0.945309  0.28801  
 0.675954   0.163926    0.444574  0.247496     0.551878   0.925864  0.85485  
 0.398149   0.833481    0.149874  0.01727      0.900518   0.140022  0.563715 
 0.860873   0.23888     0.413898  0.558556     0.155949   0.154311  0.168634 

In [34]:
@time iterate(Φ, σ = 1e-2)

  0.000703 seconds (5.04 k allocations: 749.234 KiB)


([3.4441e-26 6.79586e-8 … 1.23167e-23 0.0699578; 1.01113e-40 0.1 … 2.34157e-26 2.81518e-42; … ; 5.60238e-27 0.00348553 … 4.52713e-45 8.19049e-22; 0.0414009 3.10365e-24 … 1.11991e-39 3.37258e-34], 0.8255739777893775, 502, 9.760260581970215e-7)

In [41]:
function iterate2(Φ; σ = 1e-1, tol=1e-6)
    K = exp.(Φ/σ)
    nbx = size(Φ, 1)
    nby = size(Φ, 2)
    ux = ones(nbx)/nbx
    vy = ones(nby)/nby
    error = 1.
    iteration = 0

    #π = ones(nb, nb)*1/nb/nb
    #πold = ones(nb, nb)*1/nb/nb;

    uxold = copy(ux)
    vyold = copy(vy)

    while (error > tol)

        for x in 1:nbx
            ux[x] = - σ * log((1/nbx)) + σ * logsumexp((Φ[x, :] - vy)/σ)
        end
        
        for y in 1:nby
            vy[y] = - σ * log((1/nby)) + σ * logsumexp((Φ[:, y] - ux)/σ)
        end
        
        #vy = sum(exp.((Φ .- ux)/σ), 1)

        #πold = copy(π)

        #error = maximum(abs.(π-πold))
        error = max(maximum(abs.(ux - uxold)), maximum(abs.(vy - vyold)))
        iteration += 1
        uxold = copy(ux)
        vyold = copy(vy)
    end

    π = exp.(( Φ .- ux .- vy') / σ)
    return (π, sum(π .* Φ), iteration, error)
    
end

iterate2 (generic function with 1 method)

In [42]:
@time iterate2(Φ, σ = 1e-2)

  0.610936 seconds (132.09 k allocations: 6.101 MiB)


([3.44301e-26 6.77759e-8 … 1.22825e-23 0.0699525; 1.01362e-40 0.100008 … 2.34154e-26 2.82278e-42; … ; 5.61695e-27 0.00348631 … 4.5277e-45 8.21376e-22; 0.0414004 3.09625e-24 … 1.11713e-39 3.37335e-34], 0.8255752316470029, 165, 9.54459414392872e-7)

In [1]:
using BenchmarkTools

In [15]:
n = convert(Int64, 1e5)
a = zeros(n)
b = ones(n)

function test!(a, b, n)
    for i in 1:1000
        for j in 1:n
            a[j] += 0.0002 * b[j]
        end
    end
end

@code_native test!(a, b, n)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[15]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 7
	testq	%rdx, %rdx
	jle	L114
Source line: 8
	movq	(%rdi), %rax
	movq	24(%rdi), %r8
	movq	(%rsi), %r9
	movq	24(%rsi), %r11
	movl	$1, %r10d
	movabsq	$4741619688, %rcx       ## imm = 0x11A9F5FE8
	movsd	(%rcx), %xmm0           ## xmm0 = mem[0],zero
	nopl	(%rax,%rax)
L48:
	xorl	%ecx, %ecx
	nopw	%cs:(%rax,%rax)
L64:
	cmpq	%r8, %rcx
	jae	L119
	cmpq	%r11, %rcx
	jae	L153
	movsd	(%r9,%rcx,8), %xmm1     ## xmm1 = mem[0],zero
	mulsd	%xmm0, %xmm1
	addsd	(%rax,%rcx,8), %xmm1
	movsd	%xmm1, (%rax,%rcx,8)
Source line: 7
	incq	%rcx
	cmpq	%rcx, %rdx
	jne	L64
Source line: 6
	incq	%r10
	cmpq	$1001, %r10             ## imm = 0x3E9
	jne	L48
Source line: 8
L114:
	movq	%rbp, %rsp
	popq	%rbp
	retq
L119:
	movq	%rsp, %rax
	leaq	-16(%rax), %rsi
	movq	%rsi, %rsp
	incq	%rcx
	movq	%rcx, -16(%rax)
	movabsq	$jl_bounds_error_ints, %rax
	movl	$1, %edx
	callq	*%rax
L153:
	movq	%rsp, %rdx
	leaq	-16(%rdx), %rax
	mov

In [12]:
n = convert(Int64, 1e5)
a = zeros(n)
b = ones(n)

function test1!(a, b, n)
     for i in 1:1000
        for j in 1:n
            @inbounds a[j] += 0.0002 * b[j]
        end
    end
end

@benchmark test1!(a, b, n)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     159.013 ms (0.00% GC)
  median time:      167.455 ms (0.00% GC)
  mean time:        181.965 ms (0.00% GC)
  maximum time:     301.953 ms (0.00% GC)
  --------------
  samples:          28
  evals/sample:     1

In [13]:
n = convert(Int64, 1e5)
a = zeros(n)
b = ones(n)

function test2!(a, b, n)
    for i in 1:1000
        @simd for j in 1:n
            @inbounds a[j] += 0.0002 * b[j]
        end
    end
end

@benchmark test2!(a, b, n)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     157.067 ms (0.00% GC)
  median time:      162.189 ms (0.00% GC)
  mean time:        165.205 ms (0.00% GC)
  maximum time:     203.020 ms (0.00% GC)
  --------------
  samples:          31
  evals/sample:     1